<a href="https://colab.research.google.com/github/amaeda1967/Training/blob/master/BERT_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
pip install transformers

In [22]:
import torch
from transformers import BertTokenizer, BertModel

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
vocablist = list(tokenizer.vocab.keys())

In [27]:
len(vocablist)

30522

In [28]:
'iphone' in vocablist

True

In [29]:
text = "After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."

In [31]:
tokenized_text = tokenizer.tokenize(text)

In [33]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [34]:
segment_ids = [1]*len(tokenized_text)

In [36]:
token_tensor = torch.tensor(indexed_tokens)
segments_tensor = torch.tensor(segment_ids)

In [37]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True, output_attentions = True)

In [40]:
word_embeddings = model.embeddings.word_embeddings.weight

In [46]:
len(word_embeddings)

30522

In [47]:
indexed_tokens

[2044,
 11065,
 2769,
 2013,
 1996,
 2924,
 11632,
 1010,
 1996,
 2924,
 27307,
 2001,
 2464,
 5645,
 2006,
 1996,
 5900,
 2314,
 2924,
 1012]

In [52]:
we_array = word_emnbeddings.detach().numpy()

In [54]:
import numpy as np

In [56]:
we_array.shape

(30522, 768)

In [ ]:
king = 